In [ ]:
using Plots, LaTeXStrings
using STMO
using Images, Colors

# Motivation

# Motivating example: a party in the research group

Let's have a party in our research unit! Pastries and party hats for everyone! We ask Tinne, our laboratory manager, to make some desserts: an airy merveilleux, some delicious eclairs, a big bowl of dark chocolate mousse, a sweet passion fruit-flavored bavarois and moist carrot cake (we got to have our vegetables). If we mentally cut all these sweets into portions, we have twenty portions.

In [ ]:
staff = ["Bernard", "Jan", "Willem", "Hilde", "Steffie", "Marlies", "Tim", "Wouter"]
desserts = ["merveilleux", "eclair", "chocolate mousse", "bavarois", "carrot cake"]

Since this is academia, we respect the hierarchy: people higher on the ladder are allowed to take more dessert. The professors, Bernard, Jan and Willem each get three pieces each, our senior post-doc Hilde will take four portions (one for each of her children) and the teaching assistants are allowed two portions per person. Since Wouter is a shared teaching assistant with the Biomath research group, he can only take one (sorry Wouter).

In [ ]:
# portions per person
a = [3.0, 3, 3, 4, 2, 2, 2, 1]
# quantities of each dessert
b = [4.0, 2, 6, 4, 4]

In [ ]:
ha = bar(a, color=mygreen, title="staff");
xticks!(1:8, staff)
hb = bar(b, color=myorange, title="desserts");
xticks!(1:5, desserts)
plot(ha, hb)

As engineers and mathematicians, we pride ourselves in doing things the optimal way. So how can we divide the desserts to make everybody as happy as possible? As I am preparing a course on optimization, I went around and asked which of those treats they liked. On a scale between -2 and 2, with -2 being something they hated and 2 being their absolute favorite, the desert preferences of the teaching staff is given below (students: take note!).

In [ ]:
preferences = [2 2 1 0 0;
              0 -2 -2 -2 2;
              1 2 2 2 -1;
              2 1 0 1 -1;
              0.5 2 2 1 0;
              0 1 1 1 -1;
             -2 2 2 1 1;
              2 1 2 1 -1]

C = -preferences;  # cost

In [ ]:
heatmap(preferences, title="preferences", color=:pu_or)
yticks!(1:8, staff)
xticks!(1:5, desserts)

See how most people like eclairs and chocolate mousse, but merveilleus are a more polarizing dessert! Jan is lactose intolerant, so he only gave a high score to the carrot cake by default.

The task is clear: divide these desserts in such a way that people get their portions of the kinds they like the most!

# The Monge problem

The original version of optimal transport was formulated by Gaspard Monge in 1781. Here, $n=m$ and the goal is to connect $n$ sources with $n$ sinks to minimize a cost:
$$
\min_{\sigma\in\text{Perm(n)}} \frac{1}{n}\sum_{i=1}^nC_{i,\sigma(i)}\,,
$$
with $\text{Perm(n)}$ the set of all permutation of $n$ elements.

> **Example**: There are $n$ mines mining iron ore and a collection of $n$ factories. Given a distance between every mine and every factory, select one factory for every mine such that the total cost (=transportation distance) is minimized.

The **Monge problem** is a *discrete combinatorial optimization problem*:

- The size of the search space is $n!$, for $n=70$, there are more than $10^{100}$ permutations!
- Can be solved using the [Hungarian algorithm](https://en.wikipedia.org/wiki/Hungarian_algorithm).
- Restrictive: two sets to match must be of same size. How to deal with different weights?

For working with probability distributions, we require a way to perform *soft-matching*.

## Exercise: cell tracking

In a microscopy imaging experiment we monitor ten moving cells at time $t_1$ and some time later at time $t_2$. Between these times, the cells have moved. An image processing algorithm determined the coordinates of every cell in the two images. We want to know which cell in the first corresponds to the second image. To this end, search the assignment that minimizes the sum of the squared Euclidian distances between cells from the first image versus the corresponding cell of the second image.

1. `X1` and `X2` contain the $x,y$ coordinates of the cells for the two images. Compute the matrix $C$ containing the pairwise squared Euclidean distance. You can use the function `dist` provide by the `STMO` library.
2. Complete the function `monge_brute_force` to use brute-force search for the best permutation. You might find the function `permutations` from the `Combinatorics` library useful. How large is your search space? Time your function.
3. Make a plot connecting the cells.

In [ ]:
using STMO.OptimalTransport: X1, X2

scatter(X1[:,1], X1[:,2], color=myorange, label="location cells at t1")
xlabel!("\$x\$")
ylabel!("\$y\$")
scatter!(X2[:,1], X2[:,2], color=mygreen, label="location cells at t2")

In [ ]:
using Combinatorics: permutations

for p in permutations([1, 2, 3])
   println(p)
end

In [ ]:
function monge_brute_force(C)
  n, m = size(C)
  @assert n == m "C should be square"
  # loop over all permutations and to find the
  # matching with the lowest cost
  return best_perm, best_cost
end

In [ ]:
# get the cost matrix (i.e. pairwise squared Euclidean)
# Euclidean distances between the cells at the different times)
import STMO: dist  # Euclidean distance

C = ...

In [ ]:
# make a plot with the connections of the cells

In [ ]:
# get matching

best_perm, best_cost = monge_brute_force(C)

In [ ]:
# make a plot with the connections of the cells

# The optimal transport problem

Let us introduce some notation so we can formally state this as an optimization problem. Let $\mathbf{a}$ be the vector containing the amount of dessert every person may eat. In this case $\mathbf{a} = [3,3,3,4,2,2,2,1]^\intercal$ (in general the dimension of $\mathbf{r}$ is $n$). Similarly, $\mathbf{c}$ denotes the vector of how much there is of every dessert, i.e. $\mathbf{b}=[4, 2, 6, 4, 4]^\intercal$ (in general the dimension of $\mathbf{b}$ is $m$). Often $\mathbf{a}$ and $\mathbf{b}$ represent marginal probability distributions, hence their values are nonzero sum to one.

Let $U(\mathbf{a}, \mathbf{b})$ be the set of positive $n\times m$ matrices for which the rows sum to $\mathbf{a}$ and the columns sum to $\mathbf{b}$:

$$
U(\mathbf{a}, \mathbf{b}) = \{P\in \mathbb{R}_{>0}^{n\times m}\mid P\mathbf{1}_m = \mathbf{a}, P^\intercal\mathbf{1}_n = \mathbf{b}\}\,.
$$

For our problem, $U(\mathbf{a}, \mathbf{b})$ contains all the ways of dividing the desserts for my colleagues. Note that we assume here that we can slice every dessert however we like. We do not have only to give whole pieces of pie but can provide any fraction we want.

The preferences of each person for each dessert is also stored in a matrix. To be consistent with the literature, this will be stored in a $n\times m$ *cost* matrix $C$. The above matrix is a preference matrix that can easily be changed into a cost matrix by flipping the sign of every element.

So finally, the problem we want to solve is formally posed as

$$
d_C(\mathbf{r}, \mathbf{c}) = \min_{P\in U(\mathbf{a}, \mathbf{b})}\, \sum_{i,j}P_{ij}C_{ij}\,.
$$

This is called the *optimal transport* between $\mathbf{a}$ and $\mathbf{b}$. It can be solved relatively easily using linear programming.

The optimum, $d_C(\mathbf{a}, \mathbf{b})$, is called the *Wasserstein metric*. It is basically a distance between two probability distributions. It is sometimes also called the *earth mover distance* as it can be interpreted as how much 'dirt' you have to move to change one 'landscape' (distribution) in another.

# Choosing a bit of everything

Consider a slightly modified form of the optimal transport:

$$
d_C^\lambda(\mathbf{a}, \mathbf{b}) = \min_{P\in U(\mathbf{a}, \mathbf{b})}\, \sum_{i,j}P_{ij}C_{ij} - \frac{1}{\lambda}H(P)\,,
$$

in which the minimizer $d^\lambda_C(\mathbf{a}, \mathbf{b})$ is called the *Sinkhorn distance*. Here, the extra term

$$
H(P) = -\sum_{i,j}P_{ij}\log P_{ij}
$$

is the *information entropy* of $P$. One can increase the entropy by making the distribution more homogeneous, i.e., giving everybody a more equal share of every dessert. The parameter $\lambda$ determines the trade-off between the two terms: trying to give every person only their favorites or encouraging equal distributions. Machine learners will recognize this as similar to regularization in, for example, ridge regression. Similar to that for machine learning problems, a tiny bit of shrinkage of the parameter can lead to improved performance, the Sinkhorn distance is also observed to work better than the Wasserstein distance on some problems. This is because we use a very natural prior on the distribution matrix $P$: in the absence of a cost, everything should be homogeneous!

If you squint your eyes a bit, you can also recognize a Gibbs free energy minimization problem into this, containing energy, entropy, physical restrictions ($U(\mathbf{a}, \mathbf{b})$) and a temperature ($1/\lambda$). This could be used to describe a system of two types of molecules (for example proteins and ligands) which have a varying degree of cross-affinity for each other.

## An elegant algorithm for Sinkhorn distances

Even though the entropic regularization can be motivated, to some extent, it appears that we have made the problem harder to solve because we added an extra term. Remarkably, there exists a very simple and efficient algorithm to obtain the optimal distribution matrix $P_\lambda^\star$ and the associated $d_C^\lambda(\mathbf{a}, \mathbf{b})$! This algorithm starts from the observation that the elements of the optimal distribution matrices are of the form

$$
(P_\lambda^\star)_{ij} = \alpha_i\beta_j e^{-\lambda C_{ij}}\,,
$$

with $\alpha_1,\ldots,\alpha_n$ and $\beta_1,\ldots,\beta_n$ some constants that have to be determined such that the rows, resp. columns, sum to $\mathbf{r}$, resp. $\mathbf{c}$! The optimal distribution matrix can be obtained by the following algorithm.

> **given**: cost matrix $C$, marginals $\mathbf{a}$, $\mathbf{a}$ and $\lambda\ge0$
>
> **initialize**: $P_\lambda = e^{-\lambda C}$
>
> **repeat**
>> 1. **scale the rows** such that the row sums match $\mathbf{a}$
>> 2. **scale the columns** such that the column sums match $\mathbf{b}$
>
> **until** convergence

**Assignments**
1. Complete the code below to solve optimal transport using the Sinkhorn algorithm.
2. Solve the dessert problem, once for $\lambda=0.1$ and once for $\lambda=10$.

In [ ]:
function sinkhorn(C::Matrix, a::Vector, b::Vector; λ=1.0, ϵ=1e-8)
    n, m = size(C)
    @assert n == length(a) && m == length(b) throw(DimensionMismatch("a and b do not match"))
    @assert sum(a) ≈ sum(b) "a and b don't have equal sums"
    u, v = copy(a), copy(b)
    M = ...  # complete this
    # normalize this matrix
    while maximum(abs.(a .- Diagonal(u) * (M * v))) > ϵ
        # scale u
        # scale v
      end
    return Diagonal(u) * M * Diagonal(v)
  end

In [ ]:
# solve the dessert problem for λ=0.1

In [ ]:
# solve the dessert problem for λ=10

Using this algorithm, we can compute the optimal distribution of desserts, shown below.

![The solution of the dessert problem with $\lambda=10$, an excellent approximation of the unregularized problem.](Figures/desserts_high_lamda.png)

Here, everybody only has desserts they like. Note that for example, Jan gets three pieces of carrot cake (the only thing he can eat) while Tim receives the remaining portion (he is the only person with some fondness of this dessert). If we decrease the regularization parameter $\lambda$, we encourage a more homogeneous distribution, though some people will have to try some sweets which are not their favorites...

![The solution with a slightly lower $\lambda$. Clearly, a different optimal distribution is obtained.](Figures/desserts_low_lamda.png)

The optimal transport problem, with or without entropic regularization has a beautiful geometric interpretation, shown below.

![A geometric view of the optimal transport problem.](Figures/optimal_transport_geometric.png)

The cost matrix determines a direction in which distributions are better or worse. The set $U(\mathbf{r}, \mathbf{c})$ contains all feasible distributions. In the unregularized case, the optimum $P^\star$ is usually found in one of the corners of such a set. When adding the entropic regularizer, we restrict ourselves to distributions with a minimum of entropy, lying within the smooth red curve. Because we don't have to deal with the sharp corners of $U(\mathbf{r}, \mathbf{c})$ anymore, it is easier to find the optimum. As special cases, when $\lambda\rightarrow \infty$, then $P^\star_\lambda$ will become closers to $P^\star$ (until the algorithm runs into numerical difficulties). For $\lambda\rightarrow 0$ on the other hand, only the entropic term is taken into account and $P_\lambda^\star=\mathbf{a}\mathbf{b}^\intercal$. This is bivariate distribution where the rows and columns are independent.

# Application: color transfer

In this exercise, we will apply optimal transportation to transfer the color scheme of one image to the other. We will read two images and exact the colors of the pixels (RGB encoded). Using optimal transport, a softmatching can be obtained between the pixels of te repective images. Then, every pixel of the first image can be recomputed using a weighted sum of the pixels of the second image (or vice versa).

First, we will load two images. Feel free to use your own!

In [ ]:
using Images, Colors

# function to subsample image
subsample(image, every=8) = image[1:every:size(image,1), 1:every:size(image,2)]

In [ ]:
image1 = load("chapters/06.OptimalTransport/Figures/butterfly3.jpg")
image2 = load("chapters/06.OptimalTransport/Figures/butterfly2.jpg")

plot(
  plot(image1, title="image 1", aspect_ratio=:equal),
  plot(image2, title="image 2", aspect_ratio=:equal))

We will subsample the images, otherwise we will end up with a huge cost matrix, the dimensions determined by the total number of pixels of the images. Note that we can also randomly subsample pixels or use $k$-means clustering to reduce the problem size.

In [ ]:
image1 = subsample(image1);
image2 = subsample(image2);

Take a look at the distribution of colors!

In [ ]:
colors1 = vec(image1)
colors2 = vec(image2)

plot(
  colorscatter(colors1, title="figure 1"),
  colorscatter(colors2, title="figure 2"))

The `Colors` package contains a function `colordiff` that quantifies the perceptive difference between two colors. It it an ideal cost function!

In [ ]:
# difference between red and green
colordiff(parse(Colorant, myred), parse(Colorant, mygreen))

In [ ]:
# difference between blue and green
colordiff(parse(Colorant, myblue), parse(Colorant, mygreen))

Use this to compute the pairwise distances between all the colors!

In [ ]:
C = ...  # compute the cost function between the pixels.

Then, we can compute the transport between the two collections of pixels. Since every pixel is equally important, we give an uniform weight for each pixel, e.g., $\mathbf{a}=\mathbf{1}_n/m$ and $\mathbf{b}=\mathbf{1}_m/m$

In [ ]:
P = ...

Now, each pixel of one image is soft-matched to pixels of the other picture. Recolor each pixel based on the weighted average of its assigned pixels to do the color transfer.

In [ ]:
"""
Maps one distribution to the other
"""
mapdistr(X, P) = Diagonal(sum(P) * ones(size(P, 2)).^-1)* P * X